# Test-Driven Development with FastAPI and Docker

## Introduction

### App Overview
By the end of this course, you will have built an asynchronous text summarization API with Test-Driven Development. The API itself will follow RESTful design principles, using the basic HTTP verbs: GET, POST, PUT, and DELETE.

Along with Python and FastAPI, we'll use Docker to quickly set up our local development environment and simplify deployment. Tortoise ORM, an async ORM (Object Relational Mapper), will be used to interact with a Postgres database. We'll use pytest instead of unittest for writing unit and integration tests to test the API. Finally, we'll store the code on a GitHub repository and utilize GitHub Actions to run tests before deploying to Heroku.

Before diving in, let's take a minute to go over why some of the above tools are being used.

### FastAPI
While Flask and Django are the two most popular Python web frameworks, FastAPI has surged in popularity since 2019 since it supports async out-of-the box and has an advanced data validation feature based on Python type hints.

Put simply, FastAPI is a modern, batteries-included Python web framework that's perfect for building RESTful APIs. It can handle both synchronous and asynchronous requests and has built-in support for data validation, JSON serialization, authentication and authorization, and OpenAPI (version 3.0.2 as of writing) documentation.

Highlights:

1. Heavily inspired by Flask, it has a lightweight microframework feel with support for Flask-like route decorators.
2. It takes advantage of Python type hints for parameter declaration which enables data validation (via pydantic) and OpenAPI/Swagger documentation.
3. Built on top of Starlette, it supports the development of asynchronous APIs.
4. It's fast. Since async is much more efficient than the traditional synchronous threading model, it can compete with Node and Go with regards to performance.
5. Because it's based on and fully compatible with OpenAPI and JSON Schema, it supports a number of powerful tools, like Swagger UI.
6. It has amazing documentation.


### Docker
Docker is a container platform used to streamline application development and deployment workflows across various environments. It's used to package application code, dependencies, and system tools into lightweight containers that can be moved from development machines to production servers quickly and easily.

### Pytest
pytest is a test framework for Python that makes it easy (and fun!) to write, organize, and run tests. When compared to unittest, from the Python standard library, pytest:

1. Requires less boilerplate code so your test suites will be more readable.
2. Supports the plain assert statement, which is far more readable and easier to remember compared to the assertSomething methods -- like assertEquals, assertTrue, and assertContains -- in unittest.
3. Is updated more frequently since it's not part of the Python standard library.
4. Simplifies setting up and tearing down test state with its fixture system.
5. Uses a functional approach.

### Tortoise ORM
Tortoise ORM is a async ORM inspired by the Django ORM that's designed for ease of use. It's familiar constructs help make it easier for Python developers to switch over to the world of async.

Besides Tortoise, there are a number of other ORMs, query builders, and ODMs (Object Document Mapper) in active development that support async operations. Just keep in mind that the ecosystem is not mature yet, so things are moving quickly. Breaking changes are to be expected.

**ORMs:**

* FastAPI SQLAlchemy
* FastAPIwee
* GINO
* ORM
* ormar
* Piccolo

**Query Builders:**

* asyncpgsa
* Databases

**ODMs:**

* Beanie
* MongoEngine
* Motor
* ODMantic

### GitHub
GitHub Actions is a continuous integration and delivery (CI/CD) solution, fully integrated with GitHub. Along with it, we'll also use GitHub Packages, a package management service, to store Docker images.

There are a number of platforms that include remote version control along with both CI and package management:

1. GitLab
2. AWS (CodeCommit, CodeBuild, CodeDeploy, ECR)
3. GCP (Cloud Source Repositories, Cloud Build, Container Registry)
4. Azure (Repos, Pipelines, Container Registry)

Each of them have similar features and pricing models so you really can't go wrong with any of them. If you're used to using a different platform, feel free to continue to use it if that's your preference. Or, check your understanding by trying a new platform.

### Heroku
Heroku is a cloud Platform as a Service (PaaS) that provides hosting for web applications. They offer abstracted environments where you don't have to manage the underlying infrastructure, making it easy to manage, deploy, and scale web applications. With just a few clicks you can have your app up and running, ready to receive traffic.



## Getting Started
In this chapter, we'll set up the base project structure.

```python
$ mkdir fastapi-tdd-docker && cd fastapi-tdd-docker
$ mkdir project && cd project
$ mkdir app
$ python3.10 -m venv env
$ source env/bin/activate

(env)$ pip install fastapi==0.70.1
(env)$ pip install uvicorn==0.16.0
```

Add an `__init__.py` file to the "app" directory along with a `main.py` file. Within main.py, create a new instance of FastAPI and set up a synchronous sanity check route:

```python
# project/app/main.py


from fastapi import FastAPI

app = FastAPI()


@app.get("/ping")
def pong():
    return {"ping": "pong!"}
```

That's all you need to get a basic route up and running!

You should now have:

```python
└── project
    └── app
        ├── __init__.py
        └── main.py
 ```
 
Run the server from the "project" directory:

```python
(env)$ uvicorn app.main:app
    

INFO:     Started server process [84172]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
```

`app.main:app` tells Uvicorn where it can find the FastAPI application -- e.g., "within the 'app' module, you'll find the app, `app = FastAPI()`, in the `'main.py'` file.

Navigate to http://localhost:8000/ping in your browser. You should see:
```python
{
  "ping": "pong!"
}
```

Why did we use Uvicorn to serve up FastAPI rather than a development server?

Unlike Django or Flask, FastAPI does not have a built-in development server. This is both a positive and a negative in my opinion. On the one hand, it does take a bit more to serve up the app in development mode. On the other hand, this helps to conceptually separate the web framework from the web server, which is often a source of confusion for beginners when one moves from development to production with a web framework that does have a built-in development server.

New to ASGI? Read through the excellent [Introduction to ASGI: Emergence of an Async Python Web Ecosystem](https://florimond.dev/blog/articles/2019/08/introduction-to-asgi-async-python-web/) blog post.

**FastAPI automatically generates a schema based on the OpenAPI standard. You can view the raw JSON at http://localhost:8000/openapi.json. This can be used to automatically generate client-side code for a front-end or mobile application. FastAPI uses it along with Swagger UI to create interactive API documentation, which can be viewed at http://localhost:8000/docs:**

### Auto-reload
Let's run the app again. This time, we'll enable auto-reload mode so that the server will restart after changes are made to the code base:

```python
(env)$ uvicorn app.main:app --reload

INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [84187]
INFO:     Started server process [84189]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

Now when you make changes to the code, the app will automatically reload. Try this out.

### Config
Add a new file called `config.py` to the `"app"` directory, where we'll define environment-specific configuration variables:

```python
# project/app/config.py


import logging
import os

from pydantic import BaseSettings


log = logging.getLogger("uvicorn")


class Settings(BaseSettings):
    environment: str = os.getenv("ENVIRONMENT", "dev")
    testing: bool = os.getenv("TESTING", 0)


def get_settings() -> BaseSettings:
    log.info("Loading config settings from the environment...")
    return Settings()
```
Explanation:

In [7]:
# project/app/config.py

# logging: provides a way to log messages and events in your application
import logging
# os: provides access to operating system features such as environment variables
import os
# pydantic: library for data validation and settings management using Python type annotations
from pydantic import BaseSettings


# creates a logger object named log that uses the uvicorn logger, which is an ASGI server that runs FastAPI applications
log = logging.getLogger("uvicorn")


# This class represents the configuration settings for your application. It has two attributes: environment and testing.
class Settings(BaseSettings):
    # The environment attribute is a string that indicates the environment in which your application is running, such as 
    # 'dev', 'prod', etc. It gets its value from the environment variable ENVIRONMENT, or defaults to "dev" if not set.
    environment: str = os.getenv("ENVIRONMENT", "dev")
    # The testing attribute is a boolean that indicates whether your application is in testing mode or not. It gets 
    # its value from the environment variable TESTING, or defaults to 0 (which is equivalent to False) if not set.
    testing: bool = os.getenv("TESTING", 0)

# get_settings(): returns an instance of the Settings class. This function logs a message saying that it is loading the config settings 
# from the environment, and then calls the constructor of the Settings class.
def get_settings() -> BaseSettings:
    log.info("Loading config settings from the environment...")
    return Settings()


Here, we defined a `Settings` class with two attributes:

   1. `environment` - defines the environment (i.e., dev, stage, prod)
   2. `testing` - defines whether or not we're in test mode
   
`BaseSettings`, from `pydantic`, validates the data so that when we create an instance of `Settings`, environment and testing will have types of `str` and `bool`, respectively.

Update main.py like so:

```python
# project/app/main.py


from fastapi import FastAPI, Depends

from app.config import get_settings, Settings


app = FastAPI()


@app.get("/ping")
def pong(settings: Settings = Depends(get_settings)):
    return {
        "ping": "pong!",
        "environment": settings.environment,
        "testing": settings.testing
    }
```

Take note of settings: Settings = Depends(get_settings). Here, the Depends function is a dependency that declares another dependency, get_settings. Put another way, Depends depends on the result of get_settings. The value returned, Settings, is then assigned to the settings parameter.

If you're new to dependency injection, review the [Dependencies](https://fastapi.tiangolo.com/tutorial/dependencies/) guide from the offical FastAPI docs.

Run the server again. Navigate to http://localhost:8000/ping again. This time you should see:

```python
{
  "ping": "pong!",
  "environment": "prod",
  "testing": true
}
```

What happens when you set the `TESTING` environment variable to `foo`? Try this out. Then update the variable to 0.

With the server running, navigate to http://localhost:8000/ping and then refresh a few times. Back in your terminal, you should see several log messages for:

```python
Loading config settings from the environment...
```

Essentially, `get_settings` gets called for each request. If we refactored the config so that the settings were read from a file, instead of from environment variables, it would be much too slow.

Let's use `lru_cache` to cache the settings so `get_settings` is only called once.

Update `config.py`:

```python
# project/app/config.py


import logging
import os
from functools import lru_cache

from pydantic import BaseSettings


log = logging.getLogger("uvicorn")


class Settings(BaseSettings):
    environment: str = os.getenv("ENVIRONMENT", "dev")
    testing: bool = os.getenv("TESTING", 0)


@lru_cache()
def get_settings() -> BaseSettings:
    log.info("Loading config settings from the environment...")
    return Settings()
```

After the auto-reload, refresh the browser a few times. You should only see one `Loading config settings from the environment...` log message.

### Async Handlers
Let's convert the synchronous handler over to an asynchronous one.

Rather than having to go through the trouble of spinning up a task queue (like Celery or RQ) or utilizing threads, FastAPI makes it easy to deliver routes asynchronously. As long as you don't have any blocking I/O calls in the handler, you can simply declare the handler as asynchronous by adding the `async` keyword like so:

```python
@app.get("/ping")
async def pong(settings: Settings = Depends(get_settings)):
    return {
        "ping": "pong!",
        "environment": settings.environment,
        "testing": settings.testing
    }
```

That's it. Update the handler in your code, and then make sure it still works as expected.

Shut down the server once done. Exit then remove the virtual environment as well. Then, add a requirements.txt file to the "project" directory:

```python
fastapi==0.70.1
uvicorn==0.16.0
```

Finally, add a .gitignore to the project root:

```python
__pycache__
env
```

You should now have:
```python
├── .gitignore
└── project
    ├── app
    │   ├── __init__.py
    │   ├── config.py
    │   └── main.py
    └── requirements.txt
 ```
 Init a git repo and commit your code.